In [1]:
import logging
logging.getLogger('numba').setLevel(logging.WARNING)
%matplotlib inline
import IPython.display as ipd
import torch
import commons
import utils
import ONNXVITS_infer
from text import text_to_sequence

def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.symbols, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [2]:
!pip install matplotlib

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple


In [3]:
hps = utils.get_hparams_from_file("model/1374_epochs.pth.json")

In [4]:
net_g = ONNXVITS_infer.SynthesizerTrn(
    len(hps.symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model)
_ = net_g.eval()

_ = utils.load_checkpoint("model/1374_epochs.pth", net_g)

In [5]:
text1 = get_text("おはようございます。", hps)
stn_tst = text1
with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    sid = torch.LongTensor([0])
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

ValueError: Model requires 4 inputs. Input Feed contains 1